In [ ]:
from dataset import ZIFAR10
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import optim
from models import VGG11
import torch.nn as nn
import torch

In [ ]:
train_dataset = ZIFAR10(root='data', train=True, download=True)
test_dataset = ZIFAR10(root='data', train=False, download=True)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for image, label in train_loader:
    img = image[0].permute(1, 2, 0).numpy()
    plt.imshow(img)
    plt.show()
    break

In [ ]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = VGG11(3, 10)
model = model.to(device)

optimizer = optim.SGD(model.parameters(), weight_decay=0.0005, lr=0.001 ,momentum=0.9)
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)



In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    num_train_batches = 0
    for batch in train_loader:
        num_train_batches += 1
        
        data, target = batch
        data, target = data.float().to(device), target.long().to(device)

        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        if num_train_batches % 100 == 0:
            print(f'Epoch {epoch}, Train Loss: {loss.item()}')

    cum_loss = 0
    num_val_batches = 0
    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            num_val_batches += 1
            data, target = batch
            data, target = data.float().to(device), target.long().to(device)

            pred = model(data)
            loss = criterion(pred, target)
            cum_loss += loss.item()
 
    scheduler.step(cum_loss/len(test_loader))
    print(f"Epoch {epoch}, Validation Loss: {cum_loss/num_val_batches}")    